# 1.0 Introduction

There are discrete architectural elements from milestone models that you can use to design your convolutional neural networks. Specifically, models that have achieved state-of-the-art results for tasks like image classification use discrete architecture elements repeated multiple times, such as the VGG block in the [VGG](https://arxiv.org/abs/1409.1556) models, the inception module in the [GoogLeNet](https://arxiv.org/abs/1409.4842), and the residual module in the [ResNet](https://arxiv.org/abs/1512.03385). Once you can implement parameterized versions of these architectural elements, you can use them to design your models for computer
vision and other applications. This notebook will discover how to implement the critical architecture elements from milestone convolutional neural network models from scratch. After completing this lesson, you will know:

- How to implement a VGG module used in the VGG-16 and VGG-19 convolutional neural network models.

# 2.0 VGGnet

In our previous lesson, we discussed LeNet and AlexNet, two seminal Convolutional Neural Networks in the deep learning and computer vision literature. Simonyan and Zisserman first introduced VGGnet, (sometimes referred to as simply VGG) in their 2015 paper, [Very Deep Learning Convolutional Neural Networks for Large-Scale Image Recognition]((https://arxiv.org/abs/1409.1556)). 

> The primary contribution of their work was demonstrating that architecture with tiny (3x3) filters can be trained to increasingly higher depths (16-19 layers) and obtain state-of-the-art classification on the challenging ImageNet classification challenge.

This network is **characterized by its simplicity**, using only 3×3 convolutional layers stacked on top of each other in increasing depth. **Reducing the spatial dimensions of volumes is accomplished through the usage of max pooling**. Two fully connected layers, each with 4,096 nodes (and dropout in between), are followed by a softmax classifier.

VGG is often used today for <font color="red">transfer learning</font> (we will describe this technique later in this course) **as the network demonstrates an above-average ability to generalize to datasets it was not trained on** (compared to other network types such as GoogLeNet and ResNet). More times than not, if you are reading a publication or lab journal that applies transfer learning, it likely uses VGG as the base model.

Unfortunately, training VGG from scratch is a pain, to say the least. The network is brutally slow to train, and the network architecture weights themselves are quite large (over 500MB). Due to the depth of the network along with the fully-connected layers, the backpropagation phase is excruciatingly slow.

> References from practitioners such as [Adrian Rosebrock](https://www.pyimagesearch.com/), training VGG on eight GPUs took $\approx$ 10 days – with any less than four GPUs, training VGG from scratch will likely take prohibitively long (unless you can be very patient).

That said, it’s important as a deep learning practitioner to understand the history of deep learning, especially the concept of <font color="red">pre-training</font> and how **we later learned to avoid this expensive operation by optimizing our initialization weight functions**.

## 2.1 Implementing VGGNet

When implementing VGG, Simonyan and Zisserman tried variants of VGG that increased in depth. The figure below was extracted from their publication is and highlights their experiments. In particular, we are most interested in configurations A, B, D, and E. Looking at these architectures, you will notice two patterns:

1. The first is that the **network uses only 3×3 filters**. 
2. The second is as the depth of the network increases, the number of filters learned increases as well – to be exact, **the number of filters doubles each time max pooling is applied** to reduce volume size. 

> The notion of doubling the number of filters each time you decrease spatial dimensions is of historical importance in the deep learning literature and even a pattern you will see today.

<img width="600" src="https://drive.google.com/uc?export=view&id=19cSs4edO6rQ4ZodRzgU54Inv8P3m566K"/>

We perform this doubling of filters to ensure no single layer block is more biased than the others. Layers earlier in the network architecture have fewer filters, but their spatial volumes are also much larger, implying there is **more (spatial) data** to learn from.

However, we know that applying a max-pooling operation will reduce our spatial input volumes. 

> If we reduce the spatial volumes without increasing the number of filters, our layers become unbalanced and potentially biased, implying that layers earlier in the network may influence our output classification more than layers deeper. 

To combat this imbalance, we keep in mind the ratio of volume size to the number of filters. If we reduce the input volume size by 50-75%, we double the number of filters in the next set of CONV layers to maintain the balance.

The issue with training such deep architectures is that Simonyan and Zisserman found training VGG16 and VGG19 to be extremely challenging due to their depth. **If these architectures were randomly initialized and trained from scratch, they would often struggle to learn and gain any initial traction – the networks were too deep for basic random initialization**. Therefore, to train deeper variants of VGG, Simonyan and Zisserman came up with a clever concept called <font color="red">pre-training</font>.

> Pre-training is the practice of training smaller versions of your network architecture with fewer weight layers first and then using these converged network weights as the initializations for larger,
deeper networks.

In the case of VGG, the authors first trained configuration A, VGG11. VGG11 was able to converge to the level of reasonably low loss but not state-of-the-art accuracy worthy.

The weights from VGG11 were then used as initializations to configuration B, VGG13. The conv3-64 and conv3-128 layers (highlighted in bold in above figure) in VGG13 were randomly initialized while the remainder of the layers were copied over from the pre-trained VGG11 network. Using the initializations, Simonyan and Zisserman were able to train VGG13 successfully–but still not obtain state-of-the-art accuracy.

This pre-training pattern continued to configuration D, which we commonly know as VGG16. This time three new layers were randomly initialized while the other layers were copied over from VGG13. The network was then trained using these **warmed pre-trained up** layers, thereby allowing the randomly initialized layers to converge and learn discriminating patterns. Ultimately, VGG16 was able to perform very well on the ImageNet classification challenge.

> As a final experiment, Simonyan and Zisserman once again applied pre-training to configuration E, VGG19. This very deep architecture copied the weights from the pre-trained VGG16 architecture and then added another additional three convolutional layers. After training, it was found that VGG19 obtained the highest classification accuracy from their experiments; however, the size of the model (574MB) and the amount of time it took to train and evaluate the network, all for meager gains, made it less appealing to deep learning practitioners.

**If pre-training sounds like a painful, tedious process, that is because it is**. Training smaller variations of your network architecture and then using the converged weights as initializations to your deeper versions of the network is a clever trick; however, it requires training and tuning the
hyperparameters to N separate networks, where N is your final network architecture along with the number of previous (smaller) networks required to obtain the end model. Performing this process is
extremely time-consuming, especially for deeper networks with many fully-connected layers such as VGG.

<font color="red">The good news is that we no longer perform pre-training</font> when training very deep Convolutional Neural Networks – instead, we rely on a good initialization function. Instead of pure random weight initializations we now use [Xavier/Glorot](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) or MSRA (also known as He et al. initialization).
Through the work of both Mishkin and Mtas in their 2015 paper, [All you need is a good init](https://arxiv.org/abs/1511.06422) and He et al. in [Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification](https://arxiv.org/abs/1502.01852), we found that we can skip the pre-training phase entirely and jump directly to the deeper variations of the network architectures.

> After these papers were published, Simonyan and Zisserman re-evaluated their experiments and found that these **smarter initialization** schemes and activation functions could replicate their previous performance without the usage of tedious pre-training.

Additionally, **we recommend using batch normalization after the activation functions in the network**. 

> Apply batch normalization was not discussed in the original Simonyan and Zisserman paper, but as other lessons have discussed, batch normalization can stabilize your training and reduce the total number of epochs required to obtain a reasonably performing model.

## 2.2 The VGG Family of Networks

Two key components can characterize the VGG family of Convolutional Neural Networks:

1. All CONV layers in the network using only 3x3 filters.
2. Stacking multiple CONV => RELU layer sets (where the number of consecutive CONV => RELU layers typically increases the deeper we go) before applying a POOL operation.

In this section, we will discuss a variant of the VGGNet architecture, which we call “MiniVGGNet,” because the network is substantially more shallow than its big brother.

In [1]:
# import the necessary packages
from tensorflow.keras.applications import VGG16

model = VGG16(weights="imagenet")

553467096/553467096 [==============================] - 3s 0us/step


In [ ]:
model.summary()

In [3]:
# loop over the layers in the network and display them to the console
for (i, layer) in enumerate(model.layers):
  print("[INFO] {}\t{}".format(i, layer.__class__.__name__))

[INFO] 0	InputLayer
[INFO] 1	Conv2D
[INFO] 2	Conv2D
[INFO] 3	MaxPooling2D
[INFO] 4	Conv2D
[INFO] 5	Conv2D
[INFO] 6	MaxPooling2D
[INFO] 7	Conv2D
[INFO] 8	Conv2D
[INFO] 9	Conv2D
[INFO] 10	MaxPooling2D
[INFO] 11	Conv2D
[INFO] 12	Conv2D
[INFO] 13	Conv2D
[INFO] 14	MaxPooling2D
[INFO] 15	Conv2D
[INFO] 16	Conv2D
[INFO] 17	Conv2D
[INFO] 18	MaxPooling2D
[INFO] 19	Flatten
[INFO] 20	Dense
[INFO] 21	Dense
[INFO] 22	Dense


# 3.0 GoogLeNet

This section will study the **GoogLeNet architecture**, introduced by Szegedy et al. in their 2014 paper, [Going Deeper With Convolutions](https://arxiv.org/pdf/1409.4842.pdf). This paper is essential for two reasons. First, the model architecture is tiny compared to AlexNet and VGGNet ( 28MB for the weights themselves). The authors can obtain such a dramatic drop in network architecture size (while still increasing the depth of the overall network) by removing fully connected layers and instead of using global average pooling. Most of the weights in a CNN can be found in the dense FC layers – if these layers can be removed, the memory savings are massive.

Secondly, the Szegedy et al. paper makes usage of a network in-network or micro-architecture when constructing the overall macro-architecture. Up to this point, we have seen only sequential neural networks where the output of one network feeds directly into the next. We are now going to see micro-architectures, small building blocks used inside the rest of the architecture, where the output from one layer can split into various paths and be rejoined later.

Specifically, Szegedy et al. contributed the Inception module to the deep learning community, a building block that fits into a Convolutional Neural Network, enabling it to learn CONV layers with **multiple filter sizes**, turning the module into a multi-level feature extractor.

Micro-architectures such as Inception have inspired other significant variants, including the Residual module in [ResNet](https://arxiv.org/abs/1512.03385) and the Fire module in [SqueezeNet](https://arxiv.org/abs/1602.07360). 

## 3.1 The Inception Module (and its Variants)

Modern state-of-the-art Convolutional Neural Networks utilize **micro-architectures**, also called **network-in-network modules**, initially proposed by [Lin et al](https://arxiv.org/abs/1312.4400). I prefer the term micro-architecture better describes these modules as building blocks in the context of the overall macro-architecture (i.e., what you build and train).

Micro-architectures are tiny building blocks designed by deep learning practitioners to enable networks to learn (1) faster and (2) more efficiently, all while increasing network depth. 
> These micro-architecture building blocks are stacked, along with conventional layer types such as CONV, POOL, etc., to form the overall macro-architecture.

In 2014, Szegedy et al. introduced the Inception module. The general idea behind the Inception module is two-fold:

1. **It can be hard to decide the size of the filter you need to learn at given CONV layers**. Should they be 5x5 filters? What about 3x3 filters? Should we learn local features using 1x1 filters? Instead, why not learn them all and let the model decide? Inside the Inception module, we learn all three 5x5, 3x3, and 1x1 filters (computing them in parallel), concatenating the resulting feature maps along the channel dimension. The next layer in
the GoogLeNet architecture (which could be another Inception module) receives these concatenated, mixed filters and performs the same process. Taken as a whole, this process enables GoogLeNet to learn both local features via smaller convolutions and abstracted
features with larger convolutions – we do not have to sacrifice our level of abstraction at the expense of smaller features.
2. By learning multiple filter sizes, we can turn the module into a multi-level feature extractor. The 5x5 filters have a larger receptive size and can learn more abstract features. The 1x1 filters are, by definition, local. The 3x3 filters sit as a balance in between.

### 3.1.1 Inception

Now that we’ve discussed the motivation behind the Inception module, let’s look at the actual
module itself in Figure below (the original Inception module used in GoogLeNet).

<img width="450" src="https://drive.google.com/uc?export=view&id=1ja1dLUSBtZSxBwsbimHUbho0cEvno0Cb"/>

Specifically, take note of how the Inception module branches into four distinct paths from the input layer. The first branch in the Inception module learns a series of 1x1 local features from the input.

The second batch first applies 1x1 convolutions, not only as a form of learning local features but instead as dimensionality reduction. Larger convolutions (i.e., 3x3 and 5x5) by definition take more computation to perform. Therefore, if we can reduce the dimensionality of the inputs
to these larger filters by applying 1x1 convolutions, we can reduce the computation required by our network. Therefore, the number of filters learned in the 1x1 CONV in the second branch will always be smaller than the number of 3x3 filters learned directly afterward.

The third branch applies the same logic as the second branch, only this time to learn 5x5 filters. We once again reduce dimensionality via 1x1 convolutions, then feed the output into the 5x5 filters.

The fourth and final branch of the Inception module performs 3x3 max pooling with a stride of 1x1 – this branch is commonly referred to as the pool projection branch. Historically, models that perform pooling have demonstrated an ability to obtain higher accuracy, although we now
know through the work of Springenberg et al. in their 2014 paper, [Striving for Simplicity: The All Convolutional Net](https://arxiv.org/abs/1412.6806) that this is not necessarily true and that POOL layers can be replaced with CONV layers for reducing volume size.

In the case of Szegedy et al., this POOL layer was added simply because it was thought that they were needed for CNNs to perform reasonably. The output of the POOL is then fed into another series of 1x1 convolutions to learn local features.

Finally, all four-interception modules converge where they are concatenated together along the channel dimension. Special care is taken during the implementation (via zero paddings) to ensure the output of each branch has the same volume size, thereby allowing the outputs to be concatenated. The output of the Inception module is then fed into the next layer in the network. In practice, we often stack multiple Inception modules on top of each other before performing a pooling operation to reduce volume size.